In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import os
from IOUtils import IOUtils
from VAERSFileDownloader import updateVAERSFiles
from datetime import datetime
from DateProvider import DateProvider
from InternationalVaersCovid19Provider import getInternationalVaersCovid19, get_international_VAERSVAX_VAERSSYMPTOMS_Covid19
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile, saveLastUpdatedBatchCodeTable
from BatchCodeTablePersister import createGlobalBatchCodeTable
from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory
from HistogramFactoryAndPersister import createAndSaveGlobalHistograms
from BatchCodeTableFactory import BatchCodeTableFactory
from Column2DataframeAdder import addColumn2Dataframe
from CountriesByBatchcodeProvider import getCountryCountsByBatchcodeTable
from CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter import CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter


In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
years_from_start_of_COVID_vaccination_to_present = list(range(2020, datetime.now().year + 1))

In [ ]:
updateVAERSFiles(
        years = years_from_start_of_COVID_vaccination_to_present,
        workingDirectory = os.getcwd())

In [ ]:
international_VAERSVAX_Covid19, international_VAERSSYMPTOMS = get_international_VAERSVAX_VAERSSYMPTOMS_Covid19(years = years_from_start_of_COVID_vaccination_to_present)

In [ ]:
international_VAERSVAX_Covid19

In [ ]:
international_VAERSSYMPTOMS

In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(dataDir = 'VAERS', years = years_from_start_of_COVID_vaccination_to_present)
internationalVaersCovid19

In [ ]:
batchCodeTable = createGlobalBatchCodeTable(
    deleteEntriesWithADRsLessThanOrEqual = 2,
    minADRsForLethality = 100,
    batchCodeTableFactory = BatchCodeTableFactory(internationalVaersCovid19))
batchCodeTable

In [ ]:
IOUtils.saveDataFrameAsJson(batchCodeTable, '../docs/data/batchCodeTables/Global.json')
saveLastUpdatedBatchCodeTable(
    DateProvider().getLastUpdatedDataSource(),
    batchCodeTableHtmlFile="../docs/batchCodes.html")

In [ ]:
updateBatchCodeTableHtmlFile(batchCodeTable, batchCodeTableHtmlFile="../docs/HowBadIsMyBatch.html")

In [ ]:
symptomByBatchcodeTable = SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(
    VAERSVAX = international_VAERSVAX_Covid19[international_VAERSVAX_Covid19['VAX_LOT'].isin(batchCodeTable['Batch'])],
    VAERSSYMPTOMS = international_VAERSSYMPTOMS)
symptomByBatchcodeTable

In [ ]:
createAndSaveGlobalHistograms(symptomByBatchcodeTable, batchCodeTable)

In [ ]:
countryCountsByBatchcode = getCountryCountsByBatchcodeTable()
countryCountsByBatchcode

In [30]:
# FK-TODO: nur die Batchcodes von countryCountsByBatchcode beibehalten, die auch in batchCodeTable vorkommen

In [ ]:
barChartDescriptionTable = CountryCountsByBatchcodeTable2BarChartDescriptionTableConverter.convert2BarChartDescriptionTable(countryCountsByBatchcode)
barChartDescriptionTable

In [ ]:
barChartDescriptionTable['BAR_CHART_DESCRIPTION'].to_json('../docs/data/barChartDescriptionTable.json')

# Vaccine Distribution by Zipcode

In [ ]:
vaccineDistributionByZipcode  = pd.read_excel(
        io = 'data/Amended-22-01962-Pfizer-2022-0426-pulled-2022-0823.xlsx',
        usecols = ['PROVIDER_NAME', 'ZIPCODE_SHP', 'LOT_NUMBER', 'DOSES_SHIPPED'],
        dtype = {'DOSES_SHIPPED': 'int'})
vaccineDistributionByZipcode

In [ ]:
from VaccineDistributionByZipcodeSimplifier import VaccineDistributionByZipcodeSimplifier

vaccineDistributionByZipcode = VaccineDistributionByZipcodeSimplifier.sumDoses(vaccineDistributionByZipcode)
vaccineDistributionByZipcode

In [ ]:
OVERALL_DOSES_SHIPPED_by_LOT_NUMBER = vaccineDistributionByZipcode.groupby('LOT_NUMBER').agg(OVERALL_DOSES_SHIPPED = pd.NamedAgg(column = 'DOSES_SHIPPED', aggfunc = sum))
OVERALL_DOSES_SHIPPED_by_LOT_NUMBER

In [ ]:
vaccineDistributionByZipcode = addColumn2Dataframe(dataframe = vaccineDistributionByZipcode, column = OVERALL_DOSES_SHIPPED_by_LOT_NUMBER)
vaccineDistributionByZipcode

In [ ]:
from ADR_by_Batchcode_Table_Factory import create_ADR_by_Batchcode_Table_4USA

ADR_by_Batchcode_Table_4USA = create_ADR_by_Batchcode_Table_4USA(internationalVaersCovid19)
ADR_by_Batchcode_Table_4USA

In [ ]:
vaccineDistributionByZipcode = addColumn2Dataframe(dataframe = vaccineDistributionByZipcode, column = ADR_by_Batchcode_Table_4USA)
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode['Statistical Number of Adverse Reaction Reports'] = (vaccineDistributionByZipcode['DOSES_SHIPPED'] / vaccineDistributionByZipcode['OVERALL_DOSES_SHIPPED'] * vaccineDistributionByZipcode['Adverse Reaction Reports']).round(2)
vaccineDistributionByZipcode['Statistical Number of Adverse Reaction Reports (per 100,000)'] = (vaccineDistributionByZipcode['DOSES_SHIPPED'] / vaccineDistributionByZipcode['OVERALL_DOSES_SHIPPED'] * 100000).round().astype(int)
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode = vaccineDistributionByZipcode[['PROVIDER_NAME', 'ZIPCODE_SHP', 'LOT_NUMBER', 'DOSES_SHIPPED', 'Statistical Number of Adverse Reaction Reports', 'Statistical Number of Adverse Reaction Reports (per 100,000)']]
vaccineDistributionByZipcode

In [ ]:
vaccineDistributionByZipcode = vaccineDistributionByZipcode.rename(
    columns = {
        'PROVIDER_NAME': 'Provider',
        'ZIPCODE_SHP': 'ZIP Code',
        'LOT_NUMBER': 'Lot Number',
        'DOSES_SHIPPED': 'Doses Shipped'
    })
vaccineDistributionByZipcode

In [ ]:
# vaccineDistributionByZipcode.to_excel('tmp/Amended-22-01962-Pfizer-2022-0426-pulled-2022-0823_sumDoses.xlsx')

In [ ]:
from IOUtils import IOUtils

IOUtils.saveDataFrameAsJson(vaccineDistributionByZipcode, '../docs/data/vaccineDistributionByZipcode/VaccineDistributionByZipcode.json')